In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

#### 加载数据

In [27]:
# 读取训练集
df_train = pd.read_excel('回归预测.xlsx', sheet_name='训练集',header = None)
print(df_train.shape)
df_train.head()

(550, 32)


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,5,2,6,1,1,4,1,4,3,3,...,4,1,1,5,1,1,2,3,奥氮平,34
1,6,4,6,1,4,5,3,3,3,2,...,4,1,1,6,1,1,3,1,奥氮平,49
2,6,4,2,1,1,5,3,5,4,4,...,4,1,3,3,3,1,5,6,氟哌啶醇,86
3,5,2,5,3,2,4,1,1,3,1,...,5,1,3,4,2,3,2,3,奥氮平,49
4,3,5,5,3,2,2,4,3,1,3,...,4,1,4,5,4,2,3,2,齐拉西酮,69


In [28]:
# 读取测试集
df_valid = pd.read_excel('回归预测.xlsx', sheet_name='测试集',header = None)
print(df_valid.shape)
df_valid.head()

(137, 32)


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,4,3,5,1,1,4,4,1,1,3,...,3,1,2,6,3,1,2,2,齐拉西酮,40
1,6,5,4,3,6,6,1,1,1,1,...,6,1,4,6,4,5,1,5,喹硫平,86
2,4,3,3,1,1,5,4,3,3,4,...,4,1,3,5,3,2,2,4,利培酮,50
3,4,4,1,3,1,4,4,5,4,5,...,3,1,3,5,4,3,2,5,阿立哌唑,62
4,5,5,1,1,1,5,4,1,3,4,...,2,1,3,6,3,3,1,2,阿立哌唑,70


#### 数据预处理

In [74]:
# 查看数据缺失情况
print('训练集缺失值个数：',df_train.isnull().sum().sum())
print('测试集缺失值个数：', df_valid.isnull().sum().sum())

训练集缺失值个数： 0
测试集缺失值个数： 0


In [29]:
# LabelEncoder：将string类型的分类数据转换为int类型

# 创建 LabelEncoder 对象
label_encoder = LabelEncoder() 

# 将分类数据列进行标签编码
df_train.iloc[:, 30] = label_encoder.fit_transform(df_train.iloc[:, 30]) 
df_valid.iloc[:, 30] = label_encoder.fit_transform(df_valid.iloc[:, 30]) 

# 查看转换映射
mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Encoding Mapping:")
print(mapping)

df_train.iloc[:, 30]

Label Encoding Mapping:
{'利培酮': 0, '喹硫平': 1, '奋乃静': 2, '奥氮平': 3, '氟哌啶醇': 4, '阿立哌唑': 5, '齐拉西酮': 6}


/var/folders/hs/5m8lrcx52yb4wnw_8qdvzxmm0000gn/T/ipykernel_30496/3535584880.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_train.iloc[:, 30] = label_encoder.fit_transform(df_train.iloc[:, 30])
/var/folders/hs/5m8lrcx52yb4wnw_8qdvzxmm0000gn/T/ipykernel_30496/3535584880.py:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_valid.iloc[:, 30] = label_encoder.fit_transform(df_valid.iloc[:, 30])


0      3
1      3
2      4
3      3
4      6
      ..
545    6
546    0
547    0
548    5
549    0
Name: 30, Length: 550, dtype: int64

In [30]:
# 划分数据集
y_train = df_train.iloc [:,-1]
X_train = df_train.iloc [: ,: -1]
y_valid = df_valid.iloc [:,-1]
X_valid = df_valid.iloc [: ,: -1]

#### LightGBM

In [32]:
from lightgbm import LGBMRegressor

In [ ]:
# 定义平方和相对误差函数
def ssre(y_true, y_pred):
    # 防止除数为零，加上一个小的常数 epsilon
    epsilon = 1e-10
    
    # 计算相对误差
    relative_errors = (y_pred - y_true) / (y_true + epsilon)
    
    # 计算平方和相对误差
    ssre_value = np.sum(relative_errors**2)
    
    return ssre_value

In [47]:
# 外层 3 折交叉验证
outer_cv = KFold(n_splits=3, random_state=1, shuffle=True)

# 内层 2 折交叉验证
inner_cv = KFold(n_splits=2, random_state=1, shuffle=True)

# 创建模型
estimator = LGBMRegressor(min_child_samples=5, random_state=1)

# 定义超参数空间
param_grid = {
    'learning_rate': [0.001, 0.005, 0.01, 0.1] ,
    'n_estimators': [40, 60, 80] ,
    'num_leaves':[32, 64, 128] ,
    }

# 创建 GridSearchCV 对象
gs = GridSearchCV ( estimator = estimator,
                    param_grid = param_grid,
                    scoring = make_scorer(ssre ,greater_is_better = False),
                    cv=inner_cv)

# 进行外层交叉验证
outer_scores = cross_val_score(gs, X_train, y_train, scoring=make_scorer(ssre, greater_is_better=False),
                               cv=outer_cv)

In [106]:
### 可输出更详细的训练结果
# 外层 3 折交叉验证
outer_cv = KFold(n_splits=3, random_state=1, shuffle=True)

# 内层 2 折交叉验证
inner_cv = KFold(n_splits=2, random_state=1, shuffle=True)

# 创建模型
estimator = LGBMRegressor(min_child_samples=5, random_state=1)

# 定义超参数搜索空间
param_grid = {
    'learning_rate': [0.001, 0.005, 0.01],
    'n_estimators': [40, 60, 80],
    'num_leaves': [32, 64, 128],
}

# 创建 GridSearchCV 对象
gs = GridSearchCV(estimator=estimator,
                  param_grid=param_grid,
                  scoring=make_scorer(ssre, greater_is_better=False),
                  cv=inner_cv)

# 进行外层交叉验证
outer_results = []
outer_score_list = []
best_params_list = []

for train_index, test_index in outer_cv.split(X_train):
    X_outer_train, X_outer_test = X_train.iloc[train_index], X_train.iloc[test_index]
    y_outer_train, y_outer_test = y_train.iloc[train_index], y_train.iloc[test_index]

    # 进行内层交叉验证
    gs.fit(X_outer_train, y_outer_train)

    # 获取最佳参数的组合
    best_params = gs.best_params_

    # 在测试集上评估模型
    outer_score = ssre(y_outer_test, gs.best_estimator_.predict(X_outer_test))

    outer_results.append({'best_params': best_params, 'outer_score': outer_score})
    outer_score_list.append(outer_score)
    best_params_list.append(best_params)

# 输出外层交叉验证结果
for result in outer_results:
    print('Best Parameters:', result['best_params'])
    print('Outer Score:', result['outer_score'])
    print('---')


Best Parameters: {'learning_rate': 0.01, 'n_estimators': 60, 'num_leaves': 64}
Outer Score: 21.293849918780396
---
Best Parameters: {'learning_rate': 0.01, 'n_estimators': 80, 'num_leaves': 32}
Outer Score: 15.801122568551701
---
Best Parameters: {'learning_rate': 0.01, 'n_estimators': 80, 'num_leaves': 32}
Outer Score: 21.582532980427658
---


In [107]:
min_score_index = outer_score_list.index(min(outer_score_list))
best_params = best_params_list[min_score_index]
best_params

{'learning_rate': 0.01, 'n_estimators': 80, 'num_leaves': 32}

In [108]:
print('CV ssre: %.3f +/- %.3f' % (np.mean(outer_score_list), np.std(outer_score_list)))

CV ssre: 19.559 +/- 2.660


#### XGBoost

In [64]:
from xgboost import XGBModel

In [110]:
# 外层 3 折交叉验证
outer_cv = KFold(n_splits=3, random_state=1, shuffle=True)

# 内层 2 折交叉验证
inner_cv = KFold(n_splits=2, random_state=1, shuffle=True)

# 创建模型
reg_sk = XGBModel(learning_rate =1, verbosity =0)

# 定义超参数搜索空间
param_grid =[{'reg_sk__n_estimators':list(range(1, 10, 2)),
              'reg_sk__max_depth':[2, 3, 4] ,
              'reg_sk__learning_rate':[0.001 ,0.005 ,0.01]
            }]

# 创建 GridSearchCV 对象
gs = GridSearchCV (estimator =reg_sk,
                   param_grid=param_grid,
                   scoring =make_scorer(ssre ,greater_is_better =False),
                   cv=inner_cv)      

# 进行外层交叉验证
outer_results = []
outer_score_list = []
best_params_list = []

for train_index, test_index in outer_cv.split(X_train):
    X_outer_train, X_outer_test = X_train.iloc[train_index], X_train.iloc[test_index]
    y_outer_train, y_outer_test = y_train.iloc[train_index], y_train.iloc[test_index]

    # 进行内层交叉验证
    gs.fit(X_outer_train, y_outer_train)

    # 获取最佳参数的组合
    best_params = gs.best_params_

    # 在测试集上评估模型
    outer_score = ssre(y_outer_test, gs.best_estimator_.predict(X_outer_test))

    outer_results.append({'best_params': best_params, 'outer_score': outer_score})
    outer_score_list.append(outer_score)
    best_params_list.append(best_params)

# 输出外层交叉验证结果
for result in outer_results:
    print('Best Parameters:', result['best_params'])
    print('Outer Score:', result['outer_score'])
    print('---')

Best Parameters: {'reg_sk__learning_rate': 0.001, 'reg_sk__max_depth': 2, 'reg_sk__n_estimators': 1}
Outer Score: 26.391945481612787
---
Best Parameters: {'reg_sk__learning_rate': 0.001, 'reg_sk__max_depth': 2, 'reg_sk__n_estimators': 1}
Outer Score: 24.229715620749456
---
Best Parameters: {'reg_sk__learning_rate': 0.001, 'reg_sk__max_depth': 2, 'reg_sk__n_estimators': 1}
Outer Score: 42.98528275174692
---


In [111]:
min_score_index = outer_score_list.index(min(outer_score_list))
best_params = best_params_list[min_score_index]
print('Best_params:', best_params)
print('CV ssre: %.3f +/- %.3f' % (np.mean(outer_score_list), np.std(outer_score_list)))

Best_params: {'reg_sk__learning_rate': 0.001, 'reg_sk__max_depth': 2, 'reg_sk__n_estimators': 1}
CV ssre: 31.202 +/- 8.378


#### CatBoost

In [81]:
from catboost import CatBoostRegressor

In [84]:
# 外层 3 折交叉验证
outer_cv = KFold(n_splits=3, random_state=1, shuffle=True)

# 内层 2 折交叉验证
inner_cv = KFold(n_splits=2, random_state=1, shuffle=True)

# 创建模型
reg = CatBoostRegressor(iterations =500 , verbose=False)

# 定义超参数搜索空间
param_grid =[{'depth':[5 ,10],
              'learning_rate':[0.005 ,0.01]}]

# 创建 GridSearchCV 对象
gs = GridSearchCV (estimator =reg,
                   param_grid=param_grid,
                   scoring =make_scorer(ssre ,greater_is_better =False),
                   cv=inner_cv)      

# 进行外层交叉验证
outer_results = []
outer_score_list = []
best_params_list = []

for train_index, test_index in outer_cv.split(X_train):
    X_outer_train, X_outer_test = X_train.iloc[train_index], X_train.iloc[test_index]
    y_outer_train, y_outer_test = y_train.iloc[train_index], y_train.iloc[test_index]

    # 进行内层交叉验证
    gs.fit(X_outer_train, y_outer_train)

    # 获取最佳参数的组合
    best_params = gs.best_params_

    # 在测试集上评估模型
    outer_score = ssre(y_outer_test, gs.best_estimator_.predict(X_outer_test))

    outer_results.append({'best_params': best_params, 'outer_score': outer_score})
    outer_score_list.append(outer_score)
    best_params_list.append(best_params)

# 输出外层交叉验证结果
for result in outer_results:
    print('Best Parameters:', result['best_params'])
    print('Outer Score:', result['outer_score'])
    print('---')

Best Parameters: {'depth': 10, 'learning_rate': 0.01}
Outer Score: 17.229799971506694
---
Best Parameters: {'depth': 5, 'learning_rate': 0.01}
Outer Score: 16.27189477433079
---
Best Parameters: {'depth': 5, 'learning_rate': 0.01}
Outer Score: 20.349768951257055
---


In [85]:
min_score_index = outer_score_list.index(min(outer_score_list))
best_params = best_params_list[min_score_index]
print('Best_params:', best_params)
print('CV ssre: %.3f +/- %.3f' % (np.mean(outer_score_list), np.std(outer_score_list)))

Best_params: {'depth': 5, 'learning_rate': 0.01}
CV ssre: 17.950 +/- 1.741


#### AdaBoost

In [88]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

In [97]:
# 外层 3 折交叉验证
outer_cv = KFold(n_splits=3, random_state=1, shuffle=True)

# 内层 2 折交叉验证
inner_cv = KFold(n_splits=2, random_state=1, shuffle=True)

# Define the AdaBoostRegressor
base_regressor = DecisionTreeRegressor(max_depth=3)
ada_boost = AdaBoostRegressor(estimator=base_regressor, random_state=1)

# 定义超参数搜索空间
param_grid = {
    'n_estimators': [10, 20, 30, 40, 45, 50, 60],
    'learning_rate': [0.001 ,0.005 ,0.01, 0.1, 0.5],
    }

# 创建 GridSearchCV 对象
gs = GridSearchCV (estimator =ada_boost,
                   param_grid=param_grid,
                   scoring =make_scorer(ssre ,greater_is_better =False),
                   cv=inner_cv)      

# 进行外层交叉验证
outer_results = []
outer_score_list = []
best_params_list = []

for train_index, test_index in outer_cv.split(X_train):
    X_outer_train, X_outer_test = X_train.iloc[train_index], X_train.iloc[test_index]
    y_outer_train, y_outer_test = y_train.iloc[train_index], y_train.iloc[test_index]

    # 进行内层交叉验证
    gs.fit(X_outer_train, y_outer_train)

    # 获取最佳参数的组合
    best_params = gs.best_params_

    # 在测试集上评估模型
    outer_score = ssre(y_outer_test, gs.best_estimator_.predict(X_outer_test))

    outer_results.append({'best_params': best_params, 'outer_score': outer_score})
    outer_score_list.append(outer_score)
    best_params_list.append(best_params)
    
# 输出外层交叉验证结果
for result in outer_results:
    print('Best Parameters:', result['best_params'])
    print('Outer Score:', result['outer_score'])
    print('---')

Best Parameters: {'learning_rate': 0.01, 'n_estimators': 45}
Outer Score: 18.117781027503675
---
Best Parameters: {'learning_rate': 0.005, 'n_estimators': 50}
Outer Score: 16.23897521072333
---
Best Parameters: {'learning_rate': 0.001, 'n_estimators': 60}
Outer Score: 20.78721006585829
---


In [100]:
min_score_index = outer_score_list.index(min(outer_score_list))
best_params = best_params_list[min_score_index]
print('Best_params:', best_params)
print('CV ssre: %.3f +/- %.3f' % (np.mean(outer_score_list), np.std(outer_score_list)))

Best_params: {'learning_rate': 0.005, 'n_estimators': 50}
CV ssre: 18.381 +/- 1.866


#### 测试集上预测效果

In [134]:
# 超参数调优结果（在训练集上）
reg1 = LGBMRegressor(min_child_samples=5, learning_rate=0.01, n_estimators=80, num_leaves=32, random_state=1)
reg2 = XGBRegressor(learning_rate=0.001, max_depth=2, n_estimators=1, random_state=1)
reg3 = CatBoostRegressor(depth=5, learning_rate=0.01, iterations=500, verbose=False)
reg4 = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=3), learning_rate=0.005, n_estimators=50, random_state=1)

# 重新拟合模型（在训练集上）
reg1.fit(X_train, y_train)
reg2.fit(X_train, y_train)
reg3.fit(X_train, y_train)
reg4.fit(X_train, y_train)

# 模型预测（在测试集上）
y_pred1 = reg1.predict(X_valid)
y_pred2 = reg2.predict(X_valid)
y_pred3 = reg3.predict(X_valid)
y_pred4 = reg4.predict(X_valid)

# 效果评估，使用平方和相对误差
print('LGBMRegressor SSRE: %.3f' % ssre(y_valid, y_pred1))
print('XGBRegressor SSRE: %.3f' % ssre(y_valid, y_pred2))
print('CatBoostRegressor SSRE: %.3f' % ssre(y_valid, y_pred3))
print('AdaBoostRegressor SSRE: %.3f' % ssre(y_valid, y_pred4))

LGBMRegressor SSRE: 11.627
XGBRegressor SSRE: 12.177
CatBoostRegressor SSRE: 11.177
AdaBoostRegressor SSRE: 11.288


/Users/shiroxie/miniconda3/envs/env_test/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
